In [32]:
import findspark
findspark.init()

In [33]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PWC - Split the Column Data").master("local[*]").getOrCreate()

In [34]:
spark

## Generate Employee Street, City and Country by using Emp_Addr column

In [36]:
# read using .format method
df = spark.read.format("csv") \
                .option("header", True) \
                .option("delimiter", "|") \
                .load("file:///home/hadoop/Downloads/tableConvert.com_ll5ppr.csv")

df.show(truncate=False)

# read using .csv method
df2 = spark.read.csv("file:///home/hadoop/Downloads/tableConvert.com_ll5ppr.csv", header=True, sep="|")

df2.show(truncate=False)

+------+--------+-------------------------------+
|emp_id|emp_name|emp_addr                       |
+------+--------+-------------------------------+
|1     |Alice   |123 Main Street, New York, USA |
|2     |Bob     |456 Oak Avenue, Francisco, USA |
|3     |Carol   |789 Pine Road, Los Angeles, USA|
|4     |David   |321 Elm Lane, Chicago, USA     |
|5     |Emily   |654 Maple Drive, Miami, USA    |
+------+--------+-------------------------------+

+------+--------+-------------------------------+
|emp_id|emp_name|emp_addr                       |
+------+--------+-------------------------------+
|1     |Alice   |123 Main Street, New York, USA |
|2     |Bob     |456 Oak Avenue, Francisco, USA |
|3     |Carol   |789 Pine Road, Los Angeles, USA|
|4     |David   |321 Elm Lane, Chicago, USA     |
|5     |Emily   |654 Maple Drive, Miami, USA    |
+------+--------+-------------------------------+



In [37]:
from pyspark.sql.functions import split

df_res = df.withColumn("street", split("emp_addr", ",")[0]) \
            .withColumn("city", split("emp_addr", ",")[1]) \
            .withColumn("country", split("emp_addr", ",")[2]) \
            .drop("emp_addr")

df_res.show(truncate=False)

+------+--------+---------------+------------+-------+
|emp_id|emp_name|street         |city        |country|
+------+--------+---------------+------------+-------+
|1     |Alice   |123 Main Street| New York   | USA   |
|2     |Bob     |456 Oak Avenue | Francisco  | USA   |
|3     |Carol   |789 Pine Road  | Los Angeles| USA   |
|4     |David   |321 Elm Lane   | Chicago    | USA   |
|5     |Emily   |654 Maple Drive| Miami      | USA   |
+------+--------+---------------+------------+-------+



In [38]:
df_res.explain()

== Physical Plan ==
*(1) Project [emp_id#697, emp_name#698, split(emp_addr#699, ,, -1)[0] AS street#760, split(emp_addr#699, ,, -1)[1] AS city#765, split(emp_addr#699, ,, -1)[2] AS country#771]
+- FileScan csv [emp_id#697,emp_name#698,emp_addr#699] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/hadoop/Downloads/tableConvert.com_ll5ppr.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<emp_id:string,emp_name:string,emp_addr:string>




In [31]:
spark.stop()